# Deelstudie 2: Experimentele studie naar de impact van CTA’s op het aantal comments

**Hypothese:** Video’s met een expliciete CTA om een comment achter te laten, genereren significant meer comments dan video’s zonder die CTA

**Manier van werken:**
- Beschrijvende statistiek
- Normaliteitstoets
- Groepsverschillen
- Effectgrootte

In [ ]:
# Import libraries
import math
import itertools
import pandas as pd
import pingouin as pg
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
# General settings
sns.set_theme(palette='muted')

In [ ]:
# Import data
videos = pd.read_excel('../data/exp_videos.xlsx', index_col=0)
videos

In [ ]:
# Get info about the data
print(videos.shape)
print(videos.dtypes)

## STAP 1: Beschrijvende statistiek

### 1.1 Algemeen overzicht

In [ ]:
engagement_features = ['comments', 'views', 'likes', 'dislikes', 'shares', 'engagement']
videos[engagement_features].describe().T

In [ ]:
# Histogram engagement metrics
n_metrics = len(engagement_features)
n_cols = 2
n_rows = math.ceil(n_metrics / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
for i, var in enumerate(engagement_features):
    row, col = divmod(i, n_cols)
    sns.histplot(videos[var], kde=True, ax=axes[row, col])
    axes[row, col].set_title(f'Distributie: {var.capitalize()}')
    axes[row, col].set_xlabel(var.capitalize())
    axes[row, col].set_ylabel('Frequentie')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
for i, var in enumerate(engagement_features):
    row, col = divmod(i, n_cols)
    sns.boxplot(x=videos[var], ax=axes[row, col])
    axes[row, col].set_title(f'Boxplot: {var.capitalize()}')
    axes[row, col].set_xlabel(var.capitalize())
plt.tight_layout()
plt.show()

### 1.2 Opdeling in groepen (met/zonder CTA)

In [ ]:
# Divide videos into two groups: with CTA and without CTA
cta = videos[videos['has_CTA'] == True]
no_cta = videos[videos['has_CTA'] == False]

In [ ]:
print("Video's MET CTA")
cta[engagement_features].describe().T

In [ ]:
print("Video's ZONDER CTA")
no_cta[engagement_features].describe().T

In [ ]:
# Histogram
videos['CTA_label'] = videos['has_CTA'].map({False: 'Zonder CTA', True: 'Met CTA'})

# Plot
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
cta_palette = {'Zonder CTA': 'red', 'Met CTA': 'green'}
for i, var in enumerate(engagement_features):
    row, col = divmod(i, n_cols)
    sns.histplot(
        data=videos,
        x=var,
        hue='CTA_label',
        kde=True,
        ax=axes[row, col],
        element='step',
        common_norm=False,
        palette=cta_palette
    )
    axes[row, col].set_title(f'Distributie van {var}')
    axes[row, col].set_xlabel(var.capitalize())
    axes[row, col].set_ylabel('Frequentie')
plt.tight_layout()
plt.show()

In [ ]:
# Boxplot
videos['CTA_label'] = videos['has_CTA'].map({False: 'Zonder CTA', True: 'Met CTA'})

# Parameters
n_metrics = len(engagement_features)
n_cols = 2
n_rows = math.ceil(n_metrics / n_cols)

# Subplots genereren
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
cta_palette = {'Zonder CTA': 'red', 'Met CTA': 'green'}
for i, var in enumerate(engagement_features):
    row, col = divmod(i, n_cols)
    sns.boxplot(x='CTA_label', y=var, data=videos, ax=axes[row, col], palette=cta_palette, hue='CTA_label')
    axes[row, col].set_title(f'{var.capitalize()} per groep')
    axes[row, col].set_xlabel('CTA aanwezig')
    axes[row, col].set_ylabel(var.capitalize())
plt.tight_layout()
plt.show()

### 1.3 Tijdsevolutie van engagement metrics

In [ ]:
# Define measurement points
timepoints = ['24h', '1w', '2w', '1m', '2m', '3m']
time_labels = ['24u', '1w', '2w', '1m', '2m', '3m']

In [ ]:
def plot_time_evolution(metric: str):
    columns = [f"{metric}_{tp}" for tp in timepoints]

    # Average values per group per period
    cta_group = videos[videos['CTA_label'] == 'Met CTA'][columns].mean()
    no_cta_group = videos[videos['CTA_label'] == 'Zonder CTA'][columns].mean()

    # Plot
    plt.figure(figsize=(10, 5))
    plt.plot(time_labels, no_cta_group, marker='o', label='Zonder CTA', color='red')
    plt.plot(time_labels, cta_group, marker='o', label='Met CTA', color='green')
    plt.title(f'Tijdsevolutie van {metric}')
    plt.xlabel('Tijd sinds publicatie')
    plt.ylabel(f'Gemiddelde {metric}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_time_evolution('comments')
plot_time_evolution('views')
plot_time_evolution('likes')
plot_time_evolution('dislikes')
plot_time_evolution('shares')
plot_time_evolution('engagement_rate')

### 1.4 Vergelijking met grotere dataset

In [ ]:
videos_full = pd.read_excel('./data/videos.xlsx', index_col=0)

In [ ]:
videos.columns.difference(videos_full.columns)

In [ ]:
videos_full.columns.difference(videos.columns)

In [ ]:
def compare_detailed_time_evolution(metric: str):
    # Required columns
    columns = [f"{metric}_{tp}" for tp in timepoints]

    # Calc means
    full_mean = videos_full[columns].mean()
    exp_total = videos[columns].mean()
    exp_cta = videos[videos['CTA_label'] == 'Met CTA'][columns].mean()
    exp_nocta = videos[videos['CTA_label'] == 'Zonder CTA'][columns].mean()

    # Plots
    plt.figure(figsize=(10, 5))

    # Full dataset
    plt.plot(time_labels, full_mean, label='Volledige dataset', color='gray', linewidth=2)

    # Experiment totals
    plt.plot(time_labels, exp_total, label='Experiment – totaal', color='blue', linewidth=2)

    # CTA
    plt.plot(time_labels, exp_cta, label='Experiment – met CTA', color='green',
             linestyle='--', linewidth=2)

    # No CTA
    plt.plot(time_labels, exp_nocta, label='Experiment – zonder CTA', color='red',
             linestyle='--', linewidth=2)

    # Titles and labels
    plt.title(f'Tijdsevolutie van {metric}: vergelijking experiment vs populatie')
    plt.xlabel('Tijd sinds publicatie')
    plt.ylabel(f'Gemiddelde {metric}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
compare_detailed_time_evolution('comments')
compare_detailed_time_evolution('views')
compare_detailed_time_evolution('likes')
compare_detailed_time_evolution('dislikes')
compare_detailed_time_evolution('shares')
compare_detailed_time_evolution('engagement_rate')

In [ ]:
def plot_relative_deviation_from_full(metric: str):
    columns = [f"{metric}_{tp}" for tp in timepoints]

    # Average values per period
    full = videos_full[columns].mean()
    total = videos[columns].mean()
    with_cta = videos[videos['CTA_label'] == 'Met CTA'][columns].mean()
    without_cta = videos[videos['CTA_label'] == 'Zonder CTA'][columns].mean()

    # Calc deviation in %
    total_pct = ((total - full) / full) * 100
    cta_pct = ((with_cta - full) / full) * 100
    nocta_pct = ((without_cta - full) / full) * 100

    # Plot
    plt.figure(figsize=(10, 5))
    plt.axhline(0, color='black', linestyle='--', linewidth=1)

    plt.plot(time_labels, total_pct, label='Experiment – totaal', color='blue', marker='o')
    plt.plot(time_labels, cta_pct, label='Experiment – met CTA', color='green', marker='o', linestyle='--')
    plt.plot(time_labels, nocta_pct, label='Experiment – zonder CTA', color='red', marker='o', linestyle='--')

    plt.title(f'Procentuele afwijking t.o.v. kanaalgemiddelde – {metric}')
    plt.xlabel('Tijd sinds publicatie')
    plt.ylabel('Afwijking (%) t.o.v. videos_full')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_relative_deviation_from_full('comments')
plot_relative_deviation_from_full('views')
plot_relative_deviation_from_full('likes')
plot_relative_deviation_from_full('dislikes')
plot_relative_deviation_from_full('shares')
plot_relative_deviation_from_full('engagement_rate')

## STAP 2: Normaliteitstoets

In [ ]:
# Create subgroups
with_cta = videos[videos['CTA_label'] == 'Met CTA']['comments']
without_cta = videos[videos['CTA_label'] == 'Zonder CTA']['comments']

# Shapiro-Wilk test
shapiro_with = stats.shapiro(with_cta)
shapiro_without = stats.shapiro(without_cta)

# Results
print("Shapiro-Wilk – Met CTA")
print(f"  W-statistic: {shapiro_with.statistic:.4f}")
print(f"  p-value: {shapiro_with.pvalue:.4f}")
print(f"  Skewness: {with_cta.skew():.4f}")
print()

print("Shapiro-Wilk – Zonder CTA")
print(f"  W-statistic: {shapiro_without.statistic:.4f}")
print(f"  p-value: {shapiro_without.pvalue:.4f}")
print(f"  Skewness: {without_cta.skew():.4f}")

### Resultaten en interpretatie normaliteitstoets

Om te bepalen welke statistische toets geschikt is voor de vergelijking tussen video’s met en zonder CTA, werd de normaliteit van de verdeling van het aantal comments per groep geëvalueerd. Hiervoor werd de Shapiro-Wilk test toegepast, aangevuld met een analyse van de skewness.

**Samenvatting van de resultaten:**

- **Met CTA:**
    - W-statistic: 0.9596
    - p-value: 0.8165
    - Skewness: 0.6091

- **Zonder CTA:**
    - W-statistic: 0.8668
    - p-value: 0.2538
    - Skewness: 0.5672

**Interpretatie**

Voor beide groepen is de p-waarde van de Shapiro-Wilk test niet significant (p > 0.05), wat betekent dat er geen statistisch bewijs is voor een afwijking van normaliteit. Daarnaast liggen de skewness-waarden voor beide groepen tussen +0.5 en +1, wat wijst op een lichte tot matige rechtsscheefheid, maar nog binnen aanvaardbare grenzen (Field, 2018). Op basis van deze bevindingen kan worden geconcludeerd dat de verdelingen van het aantal comments binnen beide groepen voldoende normaal zijn om de onafhankelijke t-test toe te passen.

Gezien de beperkte steekproefomvang (n = 11) wordt daarnaast ook een niet-parametrische toets (Mann-Whitney U-test) uitgevoerd ter controle, conform de methodologische richtlijnen voor robuustheid in kleine steekproeven.

## STAP 3: Groepsverschillen

In [ ]:
# T-test
t_stat, p_val = stats.ttest_ind(with_cta, without_cta, equal_var=False)  # Welch's t-test

# Results
print("T-test – Met vs Zonder CTA")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_val:.4f}")

In [ ]:
# Mann-Whitney test
u_stat, p_mwu = stats.mannwhitneyu(with_cta, without_cta, alternative='two-sided')

# Results
print("Mann-Whitney U-test – Met vs Zonder CTA")
print(f"  U-statistic: {u_stat:.4f}")
print(f"  p-value: {p_mwu:.4f}")

### Resultaten en interpretatie van de hypothesetoetsing

Om te onderzoeken of educatieve YouTube-video’s met een expliciete Call-To-Action (CTA) significant meer comments genereren dan video’s zonder CTA, werden zowel een parametrische als een niet-parametrische toets uitgevoerd.

**Samenvatting van de resultaten:**

- **Onafhankelijke t-test (Welch):**
    - t-statistic: -0.3008
    - p-value: 0.7746

- **Mann-Whitney U-test:**
    - U-statistic: 15.5000
    - p-value: 1.0000

**Interpretatie**

Beide toetsen tonen geen significant verschil in het aantal gegenereerde comments tussen video’s met en zonder CTA. De p-waarden (respectievelijk 0.7746 en 1.0000) liggen ver boven de gebruikelijke drempel van α = 0.05, wat betekent dat er geen statistisch bewijs is dat de aanwezigheid van een CTA leidt tot een hoger gemiddeld aantal comments.

Deze bevinding staat op het eerste gezicht in contrast met de oorspronkelijke hypothese. Toch moet deze uitkomst genuanceerd worden in het licht van de beperkte steekproefgrootte (n = 11), waardoor de kans op een Type II-fout (ten onrechte niet-significant resultaat) reëel is. Bovendien werd in eerdere descriptieve analyses en tijdsevoluties wel degelijk een zichtbaar verschil waargenomen tussen video’s met en zonder CTA, wat suggereert dat het effect mogelijks bestaat, maar niet met voldoende statistische kracht kan worden aangetoond.

Om die reden is het aangewezen om aanvullend de effectgrootte te berekenen, zodat de praktische relevantie van het waargenomen verschil alsnog kan worden geëvalueerd.

## STAP 4: Effectgrootte

In [ ]:
# Compute Cohen's d (Welch variant standaard in pingouin)
cohen_result = pg.compute_effsize(with_cta, without_cta, eftype='cohen', paired=False)
print(f"Cohen's d: {cohen_result:.4f}")

In [ ]:
# Function to calculate Cliff's Delta
def cliffs_delta(x, y):
    n = 0
    for xi, yi in itertools.product(x, y):
        if xi > yi:
            n += 1
        elif xi < yi:
            n -= 1
    delta = n / (len(x) * len(y))
    return delta

# Calculate Cliff's Delta
delta = cliffs_delta(with_cta, without_cta)
print(f"Cliff's Delta: {delta:.4f}")

### Resultaten en interpretatie van de effectgroottes

Aangezien de hypothesetoetsing geen significant verschil opleverde, werden aanvullende effectgroottes berekend om de praktische relevantie van het verschil in commentgedrag tussen video’s met en zonder CTA te evalueren.

**Samenvatting van de resultaten:**

- **Cohen’s d:** -0.1942
- **Cliff’s Delta:** 0.0333

**Interpretatie**

De waarde van Cohen’s *d* bedraagt -0.1942, wat neerkomt op een klein tot verwaarloosbaar effect (Cohen, 1988). In absolute termen ligt de d-waarde onder de drempel van 0.2 die doorgaans als minimale indicatie voor een klein effect wordt beschouwd.

Cliff’s Delta bedraagt 0.0333, wat volgens de interpretatierichtlijnen van Romano et al. (2006) eveneens wijst op een verwaarloosbaar effect (< 0.11). Dit betekent dat de kans dat een video met CTA méér comments genereert dan een video zonder CTA, slechts marginaal hoger ligt dan kansniveau.

Beide effectgroottes bevestigen dat het verschil tussen de twee groepen niet alleen statistisch niet significant is, maar ook **praktisch beperkt**. In combinatie met de kleine steekproefgrootte blijft het echter belangrijk om voorzichtig te zijn met harde conclusies; een groter experiment zou nodig zijn om definitief uitsluitsel te geven over de werkelijke effectiviteit van CTA’s in dit opzicht.